# Prepare script for relax_dynamic:at_temp calculations

This Notebook is designed to call master prepare for relax_dynamic calculations for the at_temp branch. The Notebook provides a routine that allows for more dynamic control over which calculations are created:

- New calculations at higher temperatures are only prepared if the calculation at the next lowest temperature finished and the relaxed box dimensions appear to still be consistent with the original crystal family.
- Calculations are only set up for select prototype/reference families at a time to allow for different size mults based on different numbers of atoms in the unit cells.

In [1]:
import iprPy
import atomman as am
import numpy as np

In [6]:
np.arange(100, 3050, 50, dtype=int)

array([ 100,  150,  200,  250,  300,  350,  400,  450,  500,  550,  600,
        650,  700,  750,  800,  850,  900,  950, 1000, 1050, 1100, 1150,
       1200, 1250, 1300, 1350, 1400, 1450, 1500, 1550, 1600, 1650, 1700,
       1750, 1800, 1850, 1900, 1950, 2000, 2050, 2100, 2150, 2200, 2250,
       2300, 2350, 2400, 2450, 2500, 2550, 2600, 2650, 2700, 2750, 2800,
       2850, 2900, 2950, 3000])

## MISSING ALTERNATE LAMMPS_COMMANDS!!!!

In [2]:
database_name = 'master'
run_directory_name = 'master_6'

lammps_command = 'lmp_mpi'
#mpi_command = ''
np_per_runner = 1

family = 'A1--Cu--fcc'
symbols = 'Al'
sizemults = '10 10 10'

In [ ]:
database = iprPy.load_database(database_name)

### Prepare 50K relaxations

First, relaxation calculations at 50K are prepared for all relaxed_crystal records from the dynamic method with good standing.

In [3]:
# Load relaxed_crystal records
relaxed_df = database.get_records_df('relaxed_crystal', method='dynamic', standing='good',
                                     family=family,
                                     symbols=symbols)

# Skip OpenKIM potentials (for local Windows runs)
def get_mo(series):
    return series.potential_LAMMPS_key[:3] == 'MO_'
relaxed_df = relaxed_df[~relaxed_df.apply(get_mo, axis=1)]

parent_keys = relaxed_df.key.tolist()

database.master_prepare(
    styles = 'relax_dynamic:at_temp',
    np_per_runner = np_per_runner,
    run_directory = run_directory_name,
    num_pots = 900,
    lammps_command = 'lmp_mpi',
    temperature = '50',
    parent_key = parent_keys,
    sizemults = sizemults)

100%|████████████████████████████████████████████████████████████████████████████████| 450/450 [00:08<00:00, 53.65it/s]


793 potential ids found
Preparing calculation relax_dynamic branch at_temp
Using potential #s 0 to 599

109137 existing calculation records found
600 matching interatomic potentials found
73 matching atomic parents found
73 calculation combinations to check
0 new records to prepare


preparing: 0it [00:00, ?it/s]


Using potential #s 600 to 792



109137 existing calculation records found
429 matching interatomic potentials found
0 matching atomic parents found
1 invalid calculations skipped
0 calculation combinations to check



### Prepare > 50K relaxations

1. All finished relax_dynamic calculations are retrieved.
2. Loop over all temperatures every 50K from 100K to 3000K
3. Identify all finished calculations for 50K less than the target temperature.
4. Check if the dimensions of the relaxed unit cell is within rtol of the expected crystal family.
5. Prepare new relaxation calculations at the target temperature from the associated relaxed_crystal record.

In [ ]:
# Method parameters
rtol = 0.05
crystal_family = 'cubic'
temperatures = np.arange(100, 3050, 50, dtype=int)

In [ ]:
def check_crystal_family(df, crystal_family, atol=0.0, rtol=0.05):
    """
    Checks if the 'box' field of df, which should be an atomman.Box object,
    is of the indicated crystal family within rtol.
    """
    
    # Define box_parameters apply function based on crystal_family
    
    if crystal_family == 'cubic':
        def box_parameters(series):
            """Check values and build the box_parameter term for cubic systems"""
            if series.box.iscubic(atol=atol, rtol=rtol):
                a = np.mean([series.box.a, series.box.b, series.box.c])
                return f'{a} {a} {a}'
            else:
                return np.nan
    
    elif crystal_family == 'hexagonal':
        def box_parameters(series):
            """Check values and build the box_parameter term for hexagonal systems"""
            if series.box.ishexagonal(atol=atol, rtol=rtol):
                a = np.mean([series.box.a, series.box.b])
                c = series.box.c
                return f'{a} {a} {c} 90.0 90.0 120.0'
            else:
                return np.nan
            
    elif crystal_family == 'tetragonal':     
        def box_parameters(series):
            """Check values and build the box_parameter term for tetragonal systems"""
            if series.box.istetragonal(atol=atol, rtol=rtol):
                a = np.mean([series.box.a, series.box.b])
                c = series.box.c
                return f'{a} {a} {c}'
            else:
                return np.nan
    
    elif crystal_family == 'rhombohedral':
        def box_parameters(series):
            """Check values and build the box_parameter term for rhombohedral systems"""
            if series.box.isrhombohedral(atol=atol, rtol=rtol):
                a = np.mean([series.box.a, series.box.b, series.box.c])
                alpha = np.mean([series.box.alpha, series.box.beta, series.box.gamma])
                return f'{a} {a} {a} {alpha} {alpha} {alpha}'
            else:
                return np.nan
        
    elif crystal_family == 'orthorhombic':
        def box_parameters(series):
            """Check values and build the box_parameter term for orthorhombic systems"""
            if series.box.isorthorhombic(atol=atol, rtol=rtol):
                a = series.box.a
                b = series.box.b
                c = series.box.c
                return f'{a} {b} {c}'
            else:
                return np.nan
        
    elif crystal_family == 'monoclinic':
        def box_parameters(series):
            """Check values and build the box_parameter term for monoclinic systems"""
            if series.box.ismonoclinic(atol=atol, rtol=rtol):
                a = series.box.a
                b = series.box.b
                c = series.box.c
                beta = series.box.beta
                return f'{a} {b} {c} 90.0 {beta} 90.0'
            else:
                return np.nan
            
    elif crystal_family == 'triclinic':
        def box_parameters(series):
            """Check values and build the box_parameter term for triclinic systems"""
            if series.box.istriclinic(atol=atol, rtol=rtol):
                a = series.box.a
                b = series.box.b
                c = series.box.c
                alpha = series.box.alpha
                beta = series.box.beta
                gamma = series.box.gamma
                return f'{a} {b} {c} {alpha} {beta} {gamma}'
            else:
                return np.nan
            
    else:
        raise ValueError(f'unknown crystal family: {crystal_family}')
    
    # Use the apply function on the dataframe
    df['box_parameters'] = df.apply(box_parameters, axis=1)
        
    # Return only the dataframe entries with good box_parameters
    return df[df.box_parameters.notna()]

In [8]:
# Fetch all currently finished relax_dynamic:at_temp calculations
relaxed_df = database.get_records_df('calculation_relax_dynamic', branch='at_temp', status='finished',
                                     family=family)

# Create a box object from average lengths and tilts
def make_box(series):
    """Create cell Box from relaxed lengths and tilts"""
    return am.Box(lx=series.lx, ly=series.ly, lz=series.lz,
                  xy=series.xy, xz=series.xz, yz=series.yz)
relaxed_df['box'] = relaxed_df.apply(make_box, axis=1)

for temperature in temperatures:
    print(temperature)
    
    # Get finished calculations for the previous temperature
    previous_temperature = temperature - 50
    previous_df = relaxed_df[relaxed_df.temperature == previous_temperature].reset_index(drop=True)
    if len(previous_df) == 0:
        print('No good parents found')
        continue

    # Check if relaxed box is still the correct crystal family, within rtol.
    previous_df = check_crystal_family(previous_df, crystal_family, rtol=rtol)

    # Get good relaxed_crystal parent keys
    parent_keys = previous_df.parent_key.tolist()
    if len(parent_keys) == 0:
        print('No good parents found')
        continue
    print(len(parent_keys), 'good parents found')
    
    # Prepare calculations at the temperature for the good parents
    database.master_prepare(
        styles = 'relax_dynamic:at_temp',
        np_per_runner = np_per_runner,
        run_directory = run_directory_name,
        num_pots = 900,
        lammps_command = lammps_command,
        temperature = str(temperature),
        parent_key = parent_keys,
        sizemults = sizemults)

1050
6 good parents found


100%|████████████████████████████████████████████████████████████████████████████████| 450/450 [00:08<00:00, 51.40it/s]


793 potential ids found
Preparing calculation relax_dynamic branch at_temp
Using potential #s 0 to 792

109222 existing calculation records found
1029 matching interatomic potentials found
6 matching atomic parents found
6 calculation combinations to check
6 new records to prepare


preparing: 100%|#########################################################################| 6/6 [00:17<00:00,  2.99s/it]



1100
No good parents found
1150
6 good parents found


...


2100
No good parents found
2150
1 good parents found


100%|████████████████████████████████████████████████████████████████████████████████| 450/450 [00:07<00:00, 55.41it/s]


793 potential ids found
Preparing calculation relax_dynamic branch at_temp
Using potential #s 0 to 792

109256 existing calculation records found
1029 matching interatomic potentials found
1 matching atomic parents found
1 calculation combinations to check
1 new records to prepare


preparing: 100%|#########################################################################| 1/1 [00:03<00:00,  3.09s/it]


2200
No good parents found
2250
No good parents found
2300
No good parents found
2350
No good parents found
2400
No good parents found
2450
No good parents found
2500
No good parents found
2550
No good parents found
2600
No good parents found
2650
No good parents found
2700
No good parents found
2750
No good parents found
2800
No good parents found
2850
No good parents found
2900
No good parents found
2950
No good parents found
3000
No good parents found
